In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from scipy.stats import spearmanr
import seaborn as sns
from collections import Counter
from sklearn.cluster import KMeans, AgglomerativeClustering, BisectingKMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [ ]:
pca = PCA()
le = LabelEncoder()
ss = StandardScaler()
Kmeans = KMeans(2,n_init=5)
Agglomerative_Clustering = AgglomerativeClustering(2,metric="euclidean")
bisect_means = BisectingKMeans(n_clusters=2)

In [ ]:
votes = pd.read_csv('unicef_grant_votes.csv')
votes.rename(columns={'amount': 'funding_amount','created_at':'Funded_at'}, inplace=True)
votes['No_Grants_Funded'] = votes.groupby('source_wallet')['source_wallet'].transform('count')

data = pd.read_csv('query_data.csv')
data['last_trasaction_date'] = pd.to_datetime(data['last_trasaction_date'])
data['first_transaction_date'] = pd.to_datetime(data['first_transaction_date'])


In [ ]:
merged_df = pd.merge(data, votes , on='source_wallet', how='left')\
    .drop(['id'],axis =1)

merged_df['first_transaction_date'] = pd.to_datetime(merged_df['first_transaction_date'])
merged_df['last_trasaction_date'] = pd.to_datetime(merged_df['last_trasaction_date'])
merged_df['Funded_at'] = pd.to_datetime(merged_df['Funded_at'])
merged_df = merged_df.loc[merged_df['No_Grants_Funded']==1]

In [ ]:
X = merged_df.copy()

In [ ]:
X['first_transaction_date_day'] = X['first_transaction_date'].dt.day
X['first_transaction_date_month'] = X['first_transaction_date'].dt.month
X['first_transaction_date_year'] = X['first_transaction_date'].dt.year

X['Funded_at_day'] = X['Funded_at'].dt.day
X['Funded_at_month'] = X['Funded_at'].dt.month
X['Funded_at_year'] = X['Funded_at'].dt.year

X['last_trasaction_date_day'] = X['last_trasaction_date'].dt.day
X['last_trasaction_date_month'] = X['last_trasaction_date'].dt.month
X['last_trasaction_date_year'] = X['last_trasaction_date'].dt.year


X.drop(['first_transaction_date','Funded_at','last_trasaction_date','destination_wallet'],axis =1,inplace=True)

In [ ]:
X.drop_duplicates('source_wallet',inplace=True)

X_numeric = X[X.select_dtypes(include=['int64', 'float64']).columns]

X_scaled = ss.fit_transform(X_numeric)

X_scaled = pd.DataFrame(X_scaled, columns=X_numeric.columns)

In [ ]:
plt.figure(figsize=(7,5))
sns.heatmap(data = X_scaled.corr(numeric_only=True), fmt='g', linewidths=.5, cbar=True)

In [ ]:
X_pca = pca.fit_transform(X_scaled)

In [ ]:
labels1 = Kmeans.fit_predict(X_pca)
Counter(labels1)

In [ ]:
labels2 = Agglomerative_Clustering.fit_predict(X_pca)
Counter(labels2)

In [ ]:
labels3 = bisect_means.fit_predict(X_pca)
Counter(labels3)

In [ ]:
lists = [labels1, labels2,labels3]
corr_matrix = np.corrcoef(lists)
print(corr_matrix)

In [ ]:
majority_vote = []

for i, j, k in zip(labels1, labels2, labels3):
    count_i = [i, j, k].count(i)
    count_j = [i, j, k].count(j)
    count_k = [i, j, k].count(k)
    if count_i >= count_j and count_i >= count_k:
        majority_vote.append(i)
    elif count_j >= count_i and count_j >= count_k:
        majority_vote.append(j)
    else:
        majority_vote.append(k)

In [ ]:
X['Sybil_status'] = majority_vote

In [ ]:
X['Sybil_status'].value_counts()

In [ ]:
tp = X.loc[X['Sybil_status']== 1]

In [ ]:
data = tp.loc[(tp['txn_count']<=10)&(tp['No_Days_Active']<=3)]

In [ ]:
sns.heatmap(data.corr())

In [ ]:
data.to_csv('sybil_data.csv',index=False)